In [1]:
# use python2

# use only one GPU device
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
os.system('echo $CUDA_VISIBLE_DEVICES')

0

### Implement SegNet model
In unsampling steps , we don't use the orginal one, which records the index of max pooling, we just use the supplied UnSampling2D()

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

In [3]:
config = tf.ConfigProto()

In [4]:
config.gpu_options.allow_growth = True

In [5]:
set_session(tf.Session(config=config))

### use another way (pythonic) to write
Put all layers into a list

In [8]:
#from keras.models import Model
from keras import models
#from keras.layers import Conv2D, MaxPool2D, Dropout, UpSampling2D, Input, Merge
from keras.layers.core import Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization

In [7]:
#from keras.regularizers import l2

In [8]:
#weight_decay = 0.01#

In [9]:
img_w = 256
img_h = 256

In [22]:
??Convolution2D

In [16]:
encoding_layers = [ #Block1
                    Convolution2D(64, kernel_size=(3, 3), padding = 'same', input_shape=(img_w, img_h, 1)),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(64, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    MaxPooling2D(),
    
                    #Block2
                    Convolution2D(128, kernel_size=(3, 3), padding = 'same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(128, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    MaxPooling2D(),    

                    #Block3
                    Convolution2D(256, kernel_size=(3, 3), padding = 'same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(256, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(256, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    MaxPooling2D(),  

                    #Block4
                    Convolution2D(512, kernel_size=(3, 3), padding = 'same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(512, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(512, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    MaxPooling2D(), 
    
                    #Block5
                    Convolution2D(512, kernel_size=(3, 3), padding = 'same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(512, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(512, kernel_size=(3, 3), padding = 'same'), 
                    BatchNormalization(),
                    Activation('relu'),
                    MaxPooling2D(),
]

In [18]:
# create model
autoencoder = models.Sequential()
# add encoding layers into model
autoencoder.encoding_layers = encoding_layers
for l in autoencoder.encoding_layers:
    autoencoder.add(l)

In [19]:
decoding_layers = [ # upsample block 1
                    UpSampling2D(),
                    Convolution2D(512, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(512, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),    
                    Convolution2D(512, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),
    
                    # upsample block 2
                    UpSampling2D(),
                    Convolution2D(512, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(512, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),    
                    Convolution2D(256, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'), 
    
                    # upsample block 3
                    UpSampling2D(),
                    Convolution2D(256, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(256, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),    
                    Convolution2D(128, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'), 
    
                    # upsample block 4
                    UpSampling2D(),
                    Convolution2D(128, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(64, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),    

                    # upsample block 5
                    UpSampling2D(),
                    Convolution2D(64, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('relu'),
                    Convolution2D(21, kernel_size=(3,3), padding='same'),
                    BatchNormalization(),
                    Activation('softmax'),   
]

In [20]:
# add decoding layers into model
autoencoder.decoding_layers = decoding_layers
for l in autoencoder.decoding_layers:
    autoencoder.add(l)

In [21]:
# print the model
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
activation_2 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
__________

In [9]:
#input_shape = (512, 512, 3)
#img_input = Input(shape=input_shape)

In [23]:
# visualize
from keras.utils import plot_model
plot_model(autoencoder, to_file='./segnet.png', show_shapes=True, show_layer_names=True)

### segnet.png
https://github.com/amiltonwong/xl_dl/blob/master/keras_seg/segnet.png

In [10]:
"""
# FCN model
# block1 
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block1_conv1',
          kernel_regularizer=l2(weight_decay))(img_input)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block1_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block1_pool')(x)

# block2
x = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block2_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block2_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block2_pool')(x)

# block3
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block3_pool')(x)

# block4
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block4_pool')(x)

# block5
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block5_pool')(x)

# fully convolutional
x = Conv2D(filters=4096, kernel_size=(7,7), activation='relu', padding='same', name='fc1', 
           kernel_regularizer=l2(weight_decay))(x)
x = Dropout(0.5)(x)
x = Conv2D(filters=4096,kernel_size=(1,1), activation='relu', padding='same', name='fc2',
           kernel_regularizer=l2(weight_decay))(x)
x = Dropout(0.5)(x)

# segment
x = Conv2D(filters=21, kernel_size=(1,1), activation='softmax', padding='same',
           kernel_regularizer=l2(weight_decay))(x)

# upsample x 32
x = UpSampling2D(size=(32,32))(x)
"""

In [11]:
# construct final model
model = Model(inputs=img_input, outputs=x)

In [13]:
# print model details (e.g. feature map's size)
#model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0         
__________

In [28]:
??plot_model

In [14]:
# visualize
from keras.utils import plot_model
plot_model(model, to_file='./fcn.png', show_shapes=True, show_layer_names=True)

### fcn.png
https://github.com/amiltonwong/xl_dl/blob/master/keras_seg/fcn.png